<a href="https://colab.research.google.com/github/isakhammer/deep_learning_project/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np

**1. Implement functions for generating synthetic input data**


$$
f_1(p,q) = \frac{1}{2}q^2 + \frac{1}{2}p^2
$$

$$
f_2(p,q) = \frac{1}{2}q^2 
$$

$$
f_4(y) = 1 - cos(y)
$$



In [15]:

def generate_synthetic_batches(I,func = "2sqr"):
    
    batch = {} 
    
    if func == "2sqr":
        
        d_0 = 2
        batch["Y"] = np.random.uniform(high=2, low=-2, size=(d_0,I) )    
        batch["c"] = 0.5*batch["Y"][0,:]**2 + 0.5*batch["Y"][1,:]**2
        batch["c"] = batch["c"][:, np.newaxis]
        
        return batch
    
    elif func == "1sqr":
        d_0 = 1
        
        batch["Y"] = np.random.uniform(high=2, low=-2, size=(d_0,I) )
        batch["c"] = 0.5*(batch["Y"])**2
        batch["c"] = batch["c"].T
        
        return batch
    
    elif func == "1cos":
        d_0 = 1
        
        batch["Y"] = np.random.uniform(high=np.pi/3, low=-np.pi/3, size=(d_0,I) )
        batch["c"] = 1 - np.cos(batch["Y"])
        batch["c"] = batch["c"].T
        
        return batch
    
    else:
        raise Exception("Not axeped func")


**2. Implement the neural network for training approximation of Hamiltonian function**





$$
\tilde{F}(Y; \theta) = \eta((Z^{(K)})^T w + \mu \mathbf{1} ) 
$$
where 
$$
Z^{(k+1)} = Z^{(K)} + h  \sigma(W_k Z^{(k)} + b_1 ) \\
Z^{(0)} = \hat{I} Y.
$$
Here is a

$$
\hat{I} = 
\begin{bmatrix}
I_{d_0 \times d_0} \\
0
\end{bmatrix}
$$

The cost function is on the form 

$$
J(\theta) = \frac{1}{2} \lVert  \tilde{F} (Y; \theta) - c \rVert 
$$

where the gradient is 




In [ ]:
def F_tilde(Y, th, d_0, d_k, K, h):
    
    Z = {}
    dsigma = {}
    
    I_d = np.identity(d_k)[:,:d_0]
    Z[0] = I_d@Y
    
    
    Z_hat= th["W0"]@Z[0]+th["b0"]
    dsigma[0] = sigma( Z_hat, True)
    
    for k in range(K):
        Z_hat = th["W"+str(k)]@Z[k]+th["b"+str(k)]
        Z[k+1] = Z[k] + h*sigma(Z_hat, False)
        dsigma[k+1] = sigma(Z_hat, True)
    
    Upsilon = eta(Z[K].T@th["w"]+th["mu"])
    dUpsilon = eta(Z[K].T@th["w"]+th["mu"], derivative=True)
    
    return Z, Upsilon, dUpsilon, dsigma


def grad_J(c ,Y, th, d_0, d_k, K, h):
    
    I =  Y.shape[1]
    
    # Equation 5
    Z, Upsilon, dUpsilon, dsigma = F_tilde(Y, th, d_0, d_k, K, h) 
    
    # Equation 6
    J = 0.5*np.linalg.norm(Upsilon - c)**2

    # Initializing gradient
    dJ = {}

    # Equation (8)  
    dJ["mu"]   =  dUpsilon.T@(Upsilon - c)
        
    # Equation (9)
    dJ["w"] = Z[K]@((Upsilon - c)* dUpsilon)
        
    # Equation (10)
    P = np.zeros(( K+1, d_k, I))
    P[-1] = th["w"] @ ((Upsilon - c)* dUpsilon).T
        
    # Equation 11
    for k in range(K-1,-1,-1):
        P[k] = P[k+1] + h*th["W"+str(k)].T @ (dsigma[k] * P[k+1])  
    
    for k in range(0, K):
        # Equation 12
        dJ["W"+str(k)] = h*(P[k+1] * dsigma[k])@(Z[k]).T
        
        # Equation 13
        dJ["b"+str(k)] = h*(P[k+1] * dsigma[k])@np.ones((I,1))
    
    return J, dJ
